In [14]:
import numpy as np
from scipy.spatial import KDTree
import networkx as nx
import matplotlib.pyplot as plt
import open3d as o3d

In [5]:
pcd = o3d.io.read_point_cloud("room_furnitures.ply")
translation = pcd.get_min_bound()
pcd.translate(-translation)

PointCloud with 47659 points.

In [5]:
pcd = o3d.io.read_point_cloud("points.ply")

#center pt cloud to origin
translation = pcd.get_min_bound()
pcd.translate(-translation)

PointCloud with 100000 points.

In [4]:
def align_to_reference_plane(pcd):
    # Convert to numpy array
    points = np.asarray(pcd.points)
    
    # Fit plane using PCA
    center = np.mean(points, axis=0)
    covariance_matrix = np.cov(points - center, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
    
    # Ground plane normal (assuming it's the direction of smallest variance)
    normal = eigenvectors[:, 0]
    
    # Create rotation matrix to align normal with up direction [0, 0, 1]
    up = np.array([0, 0, 1])
    rotation_axis = np.cross(normal, up)
    rotation_angle = np.arccos(np.dot(normal, up))
    
    # Create rotation matrix using Rodrigues' formula
    rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(
        rotation_axis * rotation_angle
    )
    
    # Apply transformation
    pcd.rotate(rotation_matrix, center=center)
    return pcd

pcd = align_to_reference_plane(pcd)

In [9]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [5]:
R = o3d.geometry.get_rotation_matrix_from_xyz((np.pi / 2,0,-np.pi / 2))
rotated_pcd = pcd.rotate(R, center=pcd.get_center())

o3d.visualization.draw_geometries([rotated_pcd])

In [ ]:
def prune_to_k_neighbors(graph, k): 
    for node in graph.nodes(): 
        edges = [(node, neighbor, graph[node][neighbor]['weight']) 
                for neighbor in graph[node]] 
        if len(edges) > k: 
            # Sort edges by weight 
            edges.sort(key=lambda x: x[2]) 
            # Remove edges beyond k nearest 
            edges_to_remove = edges[k:] 
            graph.remove_edges_from([(e[0], e[1]) for e in edges_to_remove])

def build_radius_graph(points, radius, max_neighbors):     
    # Convert points to numpy array if not already 
    points = np.asarray(points) 
        
    # Create k-d tree 
    kdtree = KDTree(points) 
        
    # Initialize graph 
    graph = nx.Graph() 
        
    # Add nodes with position attributes 
    for i in range(len(points)): 
        graph.add_node(i, pos=points[i]) 
        
    # Query the k-d tree for all points within radius 
    pairs = kdtree.query_pairs(radius) 
        
    # Add edges to the graph with distances as weights 
    for i, j in pairs: 
        dist = np.linalg.norm(points[i] - points[j]) 
        graph.add_edge(i, j, weight=dist) 
        
    # If max_neighbors is specified, prune the graph 
    if max_neighbors is not None: 
        prune_to_k_neighbors(graph, max_neighbors) 
            
    return graph

In [20]:
points = np.asarray(pcd.points)

nn_d = np.mean(pcd.compute_nearest_neighbor_distance()[0])

graph = build_radius_graph(points, radius=nn_d*2, max_neighbors=10)

In [16]:
def analyze_components(graph): 
 
    components = list(nx.connected_components(graph)) 
     
    analysis = { 
        'num_components': len(components), 
        'component_sizes': [len(c) for c in components], 
        'largest_component_size': max(len(c) for c in components), 
        'smallest_component_size': min(len(c) for c in components), 
        'avg_component_size': np.mean([len(c) for c in components]), 
        'isolated_points': sum(1 for c in components if len(c) == 1) 
    } 
    return analysis

In [22]:
component_analysis = analyze_components(graph)
print("\nComponent Analysis:")

for metric, value in component_analysis.items():
    print(f"{metric}: {value}")


Component Analysis:
num_components: 5041
component_sizes: [5, 2, 2, 2, 3, 1, 1, 1, 2, 4, 1, 2, 3, 1, 1, 1, 4, 3, 1, 1, 1, 2, 1, 1036, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 3, 1, 2, 1, 2, 1, 4, 1, 1, 1, 1, 1, 2, 3, 3, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 4, 1, 2, 1, 1, 3, 1, 1, 2, 3, 2, 1, 1, 6, 2, 2, 1, 1, 6, 10, 1097, 2, 1, 2, 1, 2, 1, 1, 1, 1, 12886, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 6, 1, 1, 8, 1, 3, 3, 1, 1, 2, 5, 2, 1, 1, 1, 1, 2, 50294, 6, 1, 2, 6, 3, 1, 6, 1, 3, 2, 2, 10, 1, 1, 4, 1, 3, 1, 1, 3, 1, 6, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 2, 2, 3, 2, 1, 17, 13, 26, 1, 30, 220, 1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 2, 1, 2, 3, 2, 1, 1, 1, 4, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 8, 3, 1, 2, 1, 1, 7, 7, 1, 1, 1, 2, 1, 1, 2, 1, 11, 212, 1, 16, 1, 35, 11, 1, 15, 4, 1, 1, 4, 12, 1, 6, 1, 82, 2, 8, 1, 1, 1, 1, 1, 2, 1, 5, 1, 1, 27, 9, 5, 5, 4, 1, 1, 1, 3, 4, 1, 2, 1, 5158, 25, 2

In [19]:
def plot_cc_o3d(graph, points, cmap = 'tab20'): 
     
    # Get connected components 
    components = list(nx.connected_components(graph)) 
    n_components = len(components) 
     
    # Create color iterator 
    colors = plt.colormaps[cmap](np.linspace(0, 1, max(n_components, 1))) 
    rgb_cluster = np.zeros(np.shape(points)) 
     
    # Plot each component with a different color 
    for idx, component in enumerate(components): 
        if len(component)<=10: 
            rgb_cluster[list(component)] = [0, 0, 0] 
            idx-=1 
        else:                
            color = colors[idx][:3] 
            rgb_cluster[list(component)] = color 
     
    pcd_clusters = o3d.geometry.PointCloud() 
    pcd_clusters.points = o3d.utility.Vector3dVector(points) 
    pcd_clusters.colors = o3d.utility.Vector3dVector(rgb_cluster) 
    return pcd_clusters

In [24]:
pcd_cluster = plot_cc_o3d(graph, points)
pcd_cluster.estimate_normals()
o3d.visualization.draw_geometries([pcd_cluster])

## New

In [1]:
# dépendances : open3d, numpy, sklearn (pour sample), optionally faiss
import numpy as np
import open3d as o3d
from sklearn.neighbors import NearestNeighbors
import os

# ---------- 1) Lecture chunkée du .txt -> binaire .npy (plus rapide ensuite) ----------
def txt_to_npy_chunked(txt_path, npy_path, columns=7, chunk_lines=5_000_000):
    # lit en chunks pour éviter memoire pleine
    if os.path.exists(npy_path):
        print("npy déjà présent")
        return
    data_list = []
    with open(txt_path, 'r') as f:
        buf = []
        for i, line in enumerate(f):
            parts = line.strip().split()
            if len(parts) < 3:
                continue
            vals = [float(x) for x in parts[:columns]]
            buf.append(vals)
            if (i+1) % chunk_lines == 0:
                data_list.append(np.array(buf, dtype=np.float32))
                buf = []
        if buf:
            data_list.append(np.array(buf, dtype=np.float32))
    # concat et sauvegarde binaire
    data = np.vstack(data_list)
    np.save(npy_path, data)
    print("sauvegardé:", npy_path)

# ---------- 2) Charger binaire, convertir en open3d, voxel downsample ----------
def voxel_downsample_from_npy(npy_path, voxel_size=0.05):
    data = np.load(npy_path, mmap_mode='r')  # memory map pour économie RAM
    xyz = data[:, :3].astype(np.float32)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    down = pcd.voxel_down_sample(voxel_size=voxel_size)
    return down, data  # on garde data pour restituer attributs si besoin

# ---------- 3) Estimation courbure sur les points downsampled ----------
def compute_curvature_on_pcd(pcd, k=30):
    pts = np.asarray(pcd.points)
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(pts)
    distances, indices = nbrs.kneighbors(pts)
    curv = np.zeros(len(pts), dtype=np.float32)
    for i in range(len(pts)):
        neigh = pts[indices[i]]
        cov = np.cov(neigh.T)
        eig = np.linalg.eigvalsh(cov)
        eig = np.sort(eig)
        s = eig.sum()
        if s <= 0:
            curv[i] = 0.0
        else:
            curv[i] = eig[0] / s
    return curv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
# ---------- 4) Pipeline principal ----------
txt_path = "input.txt"
npy_path = "cloud.npy"
txt_to_npy_chunked(txt_path, npy_path, columns=7, chunk_lines=2_000_000)

# downsample (choisir voxel en fonction de densité)
down, data = voxel_downsample_from_npy(npy_path, voxel_size=0.05)
print("downsampled to", np.asarray(down.points).shape)

# estimate curvature on downsampled
curv = compute_curvature_on_pcd(down, k=30)
keep_mask = curv > 0.02  # seuil à ajuster
kept = down.select_by_index(np.where(keep_mask)[0])
o3d.io.write_point_cloud("kept_voxel_centroids.ply", kept)


npy déjà présent
downsampled to (2176687, 3)


True

In [10]:
cloud = o3d.io.read_point_cloud("kept_voxel_centroids.ply")
o3d.visualization.draw_geometries([cloud])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [6]:
down = pcd.voxel_down_sample(voxel_size=0.05)

curv = compute_curvature_on_pcd(down, k=30)
keep_mask = curv > 0.02
kept = down.select_by_index(np.where(keep_mask)[0])
o3d.io.write_point_cloud("kept_voxel_centroids.ply", kept)
cloud = o3d.io.read_point_cloud("kept_voxel_centroids.ply")

In [7]:
o3d.visualization.draw_geometries([cloud])

In [15]:
points = np.asarray(cloud.points)

nn_d = np.mean(cloud.compute_nearest_neighbor_distance()[0])

graph = build_radius_graph(points, radius=nn_d*2, max_neighbors=10)

In [17]:
component_analysis = analyze_components(graph)
print("\nComponent Analysis:")

for metric, value in component_analysis.items():
    print(f"{metric}: {value}")


Component Analysis:
num_components: 38062
component_sizes: [3, 53153, 27287, 21170, 1, 9805, 19, 19, 4619, 6736, 222, 60, 11269, 233, 8376, 48686, 21, 25748, 4, 128884, 106, 3381, 24078, 116, 12020, 3641, 656, 66, 7306, 60, 4006, 35063, 3785, 18474, 3828, 14313, 127, 41, 3924, 2902, 4, 40, 5744, 987, 11449, 84, 2557, 1, 263, 6, 2, 538, 796, 6071, 338, 400, 1979, 38, 66, 89, 177, 761, 2002, 410, 21783, 27, 1623, 168, 8555, 5119, 18384, 3, 1358, 3, 4727, 66, 634, 372, 923, 8221, 707, 22, 153, 205, 23, 771, 1, 2, 97, 37, 30, 5, 154, 150, 829, 2110, 79, 212, 177, 3, 2153, 129, 182, 190, 20, 543, 8, 370, 506, 286, 577, 630, 5533, 180, 166, 95, 1910, 1383, 4674, 54, 1891, 344, 29962, 4, 42, 224, 91, 4288, 509, 28, 29, 25, 53, 24062, 4, 80, 2, 1105, 1207, 436, 2385, 1450, 329, 343, 23, 31, 2, 308, 9, 2020, 593, 3036, 7, 43, 28, 1088, 59, 4389, 10, 115, 144, 806, 112, 631, 39, 4, 573, 904, 140, 144, 22, 240, 450, 25, 2, 992, 110, 125, 347, 818, 14, 156, 3, 910, 1, 1559, 575, 59, 753, 1, 304, 

In [20]:
pcd_cluster = plot_cc_o3d(graph, points)
pcd_cluster.estimate_normals()
o3d.visualization.draw_geometries([pcd_cluster])